<a href="https://colab.research.google.com/github/EunSik312/lecture-deeplearning/blob/main/7%EC%9B%9417%EC%9D%BCYOLOv8_vidio_detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#yolov8

In [ ]:
!pip install ultralytics

from google.colab import files
from ultralytics import YOLO # COCO 사전 훈련된 YOLOv8n 모델 로드
model = YOLO("yolov8n.pt")# 모델 정보 표시 (선택사항)
model.info() # COCO8 예제 데이터셋으로 100 에포크 훈련
results = model.train(data="coco8.yaml", epochs=10, imgsz=640) # 사진 업로드하고 경로 설정
uploaded = files.upload()
image_path = list(uploaded.keys())[0] # 업로드한 이미지에 대해 YOLOv8n 모델로 추론 실행
results = model(image_path)
results[0].show()






#유튜브 다운로드 코드 이용

In [ ]:
!pip install yt-dlp ultralytics opencv-python

from google.colab import files
from ultralytics import YOLO
import yt_dlp
import os
import cv2
from IPython.display import Video, display
model = YOLO("yolov8n.pt")
model.info()
results = model.train(data="coco8.yaml", epochs=10, imgsz=640)

# 유튜브 영상 다운로드 함수
def download_youtube_video(url):
    """유튜브 영상을 다운로드하는 함수"""
    output_path = "./downloads"
    os.makedirs(output_path, exist_ok=True)

    ydl_opts = {
        'format': 'best[height<=720]',  # 720p 이하 최고 화질
        'outtmpl': os.path.join(output_path, '%(title)s.%(ext)s'),
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=False)
            title = info.get('title', 'video')

            # 영상 다운로드
            ydl.download([url])

            # 다운로드된 파일 경로 찾기
            for file in os.listdir(output_path):
                if file.endswith(('.mp4', '.mkv', '.webm')):
                    video_path = os.path.join(output_path, file)
                    print(f"다운로드 완료: {video_path}")
                    return video_path

    except Exception as e:
        print(f"다운로드 오류: {e}")
        return None

# 영상 처리 함수
def process_video(video_path, output_path="./output_video.mp4"):
    """영상에서 객체 탐지를 수행하는 함수"""
    cap = cv2.VideoCapture(video_path)

    # 영상 속성 가져오기
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"영상 정보: {width}x{height}, FPS: {fps}, 총 프레임: {total_frames}")

    # 비디오 작성기 설정
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0

    # 프레임별 처리
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # YOLO 객체 탐지 (기존 코드와 동일한 방식)
        results = model(frame)

        # 탐지 결과를 프레임에 그리기
        annotated_frame = results[0].plot()

        # 프레임 저장
        out.write(annotated_frame)

        # 진행률 표시
        if frame_count % 30 == 0:
            progress = (frame_count / total_frames) * 100
            print(f"처리 진행률: {progress:.1f}%")

        frame_count += 1

    # 자원 해제
    cap.release()
    out.release()
    cv2.destroyAllWindows()

    return output_path

# 방법 1: 유튜브 URL 직접 입력
youtube_url = input("유튜브 영상 URL을 입력하세요: ")

if youtube_url:
    # 유튜브 영상 다운로드
    video_path = download_youtube_video(youtube_url)

    if video_path:
        # 다운로드한 영상에 대해 YOLOv8n 모델로 추론 실행
        output_video = process_video(video_path)
    else:
        print("영상 다운로드에 실패했습니다.")

print("\n처리 완료!")

##위 코드 최적화(같은 기능 코드 줄이기)

In [ ]:
!pip install yt-dlp ultralytics opencv-python

from ultralytics import YOLO
import yt_dlp
import cv2
import os

# 모델 로드 (훈련 제거)
model = YOLO("yolov8n.pt")

# Vehicle 클래스 ID (COCO 데이터셋 기준)
VEHICLE_CLASSES = [2, 3, 5, 7]  # car, motorcycle, bus, truck

def download_and_process_video(url, output_path="output_video.mp4"):
    """YouTube 영상 다운로드 및 vehicle detection 통합 함수"""

    # 1. YouTube 영상 다운로드
    ydl_opts = {
        'format': 'best[height<=720]',
        'outtmpl': 'temp_video.%(ext)s',
        'quiet': True
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

    # 다운로드된 파일 찾기
    video_path = None
    for file in os.listdir('.'):
        if file.startswith('temp_video.'):
            video_path = file
            break

    if not video_path:
        print("다운로드 실패")
        return None

    # 2. 영상 처리
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # 비디오 작성기
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0

    # 프레임별 처리 (매 3프레임마다 처리로 속도 향상)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 속도 최적화: 매 3프레임마다만 detection 수행
        if frame_count % 3 == 0:
            # Vehicle만 필터링하여 detection
            results = model(frame, classes=VEHICLE_CLASSES, verbose=False)
            annotated_frame = results[0].plot()
        else:
            annotated_frame = frame

        out.write(annotated_frame)

        # 진행률 표시 (매 60프레임마다)
        if frame_count % 60 == 0:
            print(f"진행률: {frame_count/total_frames*100:.1f}%")

        frame_count += 1

    # 정리
    cap.release()
    out.release()
    os.remove(video_path)  # 임시 파일 삭제

    print(f"완료! 결과 파일: {output_path}")
    return output_path

# 실행
url = input("YouTube URL: ")
if url:
    download_and_process_video(url)

#영상 업로드 코드

In [ ]:
# YOLOv8 설치
!pip install ultralytics

# 필요한 라이브러리
from google.colab import files
from ultralytics import YOLO
import cv2
import os

# 모델 로드 (YOLOv8n - 가장 가벼운 버전)
model = YOLO("yolov8n.pt")
model.info()

# 🔽 영상 파일 업로드
uploaded = files.upload()

# 업로드된 파일명 가져오기
video_path = list(uploaded.keys())[0]

# 🔽 동영상 열기
cap = cv2.VideoCapture(video_path)

# 결과 저장을 위한 디렉토리 생성
output_path = "output.avi"
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS)

# 🔽 출력 영상 파일 설정 (코덱: XVID)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # YOLOv8 모델로 프레임 추론
    results = model.predict(frame, verbose=False)

    # 결과 이미지 얻기
    annotated_frame = results[0].plot()

    # 프레임 쓰기
    out.write(annotated_frame)

# 자원 해제
cap.release()
out.release()

# Google Colab에서 결과 영상 다운로드할 수 있게 하기
from IPython.display import HTML
from base64 import b64encode

mp4 = open(output_path, 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=640 controls>
  <source src="{data_url}" type="video/mp4">
</video>
""")


# 김영빈님 코드 복사

In [ ]:
def play_and_save_youtube_with_yolo(youtube_url, skip_frames=5, output_filename="output_with_yolo.mp4", max_duration=10):
    """
    max_duration: 최대 처리 시간(초)
    """
    ydl_opts = {
        'format': 'mp4/best[height<=480]',
        'outtmpl': tempfile.gettempdir() + '/temp_video.%(ext)s',
        'quiet': True,
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            print("⬇️ 유튜브 영상 다운로드 중...")
            info = ydl.extract_info(youtube_url, download=True)
            video_path = ydl.prepare_filename(info)

            print(f"🎥 영상 제목: {info['title']}")

            cap = cv2.VideoCapture(video_path)
            fps = cap.get(cv2.CAP_PROP_FPS)
            frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

            max_frames = int(fps * max_duration)
            print(f"⏱ 최대 {max_duration}초 (약 {max_frames} 프레임) 처리")

            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_filename, fourcc, fps / skip_frames, (frame_width, frame_height))

            frame_num = 0
            processed_frames = 0

            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                if frame_num >= max_frames:
                    break

                if frame_num % skip_frames == 0:
                    results = model.predict(frame, imgsz=640, verbose=False)[0]
                    annotated_frame = results.plot()
                    out.write(annotated_frame)

                    clear_output(wait=True)
                    plt.figure(figsize=(10, 6))
                    plt.imshow(cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB))
                    plt.title(f"YOLOv8 추론 프레임 ({frame_num})")
                    plt.axis('off')
                    plt.show()

                    time.sleep(0.1)

                frame_num += 1

            cap.release()
            out.release()
            os.remove(video_path)
            print(f"✅ 영상 추론 종료, 저장 파일: {output_filename}")

    except Exception as e:
        print(f"❌ 오류 발생: {e}")

# 실행
youtube_url = input("유튜브 영상 URL 입력: ")
play_and_save_youtube_with_yolo(youtube_url, skip_frames=5, max_duration=10)


# 양근영님 코드

In [ ]:
!pip install yt-dlp

# 원하는 유튜브 영상 다운로드
!yt-dlp -f bestvideo+bestaudio --merge-output-format mp4 https://youtube.com/shorts/YHpRZQTJkIA?si=DyMet292HtaQ2Zfc

# 다운로드된 mp4 파일 확인
for file in os.listdir("/content"): # /content 폴더에 있는 모든 파일 목록을 가져옴 (Colab 작업 디렉토리)
    if file.endswith(".mp4"): # 파일 이름이 .mp4로 끝나는지 확인 (즉, 동영상 파일인지 검사)
        print("📁 다운로드된 영상:", file)

# 모델 로드 (YOLOv8n - 경량 모델)
model = YOLO("yolov8n.pt")

# 영상 업로드
video_path = "https://youtube.com/shorts/YHpRZQTJkIA?si=DyMet292HtaQ2Zfc"

# 저장 경로 및 비디오 캡처 설정
cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# 결과 저장 경로
output_path = "result_video_youtube.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# 프레임별로 추론 및 저장
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # YOLOv8 추론
    results = model(frame, verbose=False)

    # 결과 이미지 (bounding box 포함)
    annotated_frame = results[0].plot()
    out.write(annotated_frame)

cap.release()
out.release()

# 결과 영상 표시
Video(output_path, embed=True)